<a href="https://colab.research.google.com/github/MariiaIvonina/QC_Demo/blob/main/Latest_version_HFvsDFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 紹介 #

この Jupyter Notebook は、九州大学の  Mariia IVONINA 学術研究員 が 多田 朋史 教授  による「量子計算基礎」コースの練習のために作成しました。この練習では、使用者はpythonコードを実行して、ハートリーフォック (Hartree-Fock: HF)と密度汎関数法 (Density Functional Theory: DFT)レベルの量子化学計算を実行することができます。

---

**📋指示:** pythonコードありセルを選択した状態で左にある再生ボタン▶️をクリックして、セルを順次実行してください。セルのパラメータをリセットするには、もう一度実行してください。

*   「**0. pythonパッケージをインストールする**」セルを最初から一度だけ実行すれば十分です。

*   「**1. 分子を選ぶ**」セルで分子を選んでください。オプション「`Custom molecule`」を選択し、SMILES(Simplified molecular-input line-entry system)の定義を選択すると、希望する分子を選択することができます。SMILESの定義はインターネットで検索してください。この例では、SMILESはアデニン分子に対応しております。

*    「**2. SCF 計算**」セルで基底関数系と計算方法を選択んでください。「`HF-SCFを実行する`」または「`DFT(LDA)-SCFを実行する`」のボタンで計算を実行した後、「`分子特性`」のボタンで双極子モーメントと原子の電荷を計算できます。新しい計算を実行したい場合は、このセルとそれに続くすべてのセルを再度実行してください。SCFは反復的な手続きであるため、実行出力では収束に達するまでに何サイクル行われたかを見ることができます。

*    「**3. 分子軌道のダイアグラム**」セルで最大8つ(HOMO-3 から LUMO+3 まで)フロンティア分子軌道(MOs: Molecular Orbitals)のダイアグラムを作ります。フロンティア分子軌道とは、化学反応に最も関与する分子軌道のことです。\
最高被占軌道(HOMO: Highest Occupied Molecular Orbital): これは、電子が存在する最もエネルギーの高い被占軌道です。求電子化学反応では、HOMOから電子が供給されることが多いです。\
最低空軌道(LUMO: Lowest Unoccupied Molecular Orbital): これは、電子が存在しない最もエネルギーの低い空軌道です。求核化学反応では、LUMOに電子が供給されることが多いです。\
HOMOとLUMOのエネルギー差(HOMO-LUMOギャップ)は、分子の安定性や反応性を示す指標となります。

*    「**4. Cubeファイルの生成**」セルでフロンティアMOsの空間情報を含む .cube ファイルが生成されます。

*    「**5. 3D分子軌道のプロット**」セルでフロンティアMOsを3Dで可視化することができます。\
おすすめのアイソバリューは (H2 : 0.06, water : 0.03, methane : 0.04, benzene : 0.03). 他の分子とは、最適なアイソバリューを自分で探してみてください。\
.png画像を保存したい場合は、「`show_noninteractive_png`」をチェックしてください。


In [ ]:
# @title **0. pythonパッケージをインストールする** { display-mode: "form" }

import traceback
UCBOLD = "\33[1m"
UCEND = "\033[0m"
UCGREEN, UCRED = "\33[32m", "\33[31m"
good_install = UCBOLD + "インストール・成功" + UCGREEN + " ✓" + UCEND
fail_install = UCBOLD + "インストール・エラー" + UCRED + " ✗" + UCEND + ":\n"

try:
    print("パッケージのインストール中 \u231B ...")
    !pip install pyscf
    !pip install rdkit
    !pip install py3Dmol
    !pip install -q git+https://github.com/funkymunkycool/Cube-Toolz.git

    import numpy as np
    import pandas as pd
    from pyscf import gto, scf, dft, tools
    import ipywidgets as widgets

    from rdkit import Chem, RDLogger
    RDLogger.DisableLog('rdApp.*')
    from rdkit.Chem import AllChem, rdCoordGen
    from rdkit.Chem.Draw import IPythonConsole, MolToImage
    import py3Dmol
    import cube_tools
    import matplotlib.pyplot as plt

    from IPython.display import Markdown, display, clear_output
    from google.colab import output
    import sys
    print(good_install)
except Exception:
    print(fail_install)
    print(traceback.format_exc())

パッケージのインストール中 ⌛ ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
インストール・成功 ✓


# 背景理論 #

システムの一般的なハミルトニアンは、原子単位系での相互作用する電子と原子核のシステムにおいて次のように表されます:

$$ \hat{H} = \hat{T}_{\text{e}} + \hat{T}_{\text{N}} + \hat{V}_{\text{eN}} + \hat{V}_{\text{NN}} + \hat{V}_{\text{ee}}  $$

$$  = -\frac{1}{2} \sum_{i}^{\text{elec}} \nabla^2_i -\frac{1}{2M_A} \sum_{A}^{\text{nuc}} \nabla^2_A - \sum_{i}^{\text{elec}} \sum_{A}^{\text{nuc}} \frac{Z_A}{\boldsymbol{r}_{iA}} + \frac{1}{2} \sum_{A > B}^{\text{nuc}} \frac{Z_A Z_B}{\boldsymbol{R}_{AB}}  + \frac{1}{2} \sum_{i>j}^{\text{elec}} \frac{1}{\boldsymbol{r}_{ij}}  $$


ここで、$i,j$ は電子のインデックス、$A,B$ は原子核のインデックスです。ハミルトニアンは、原子核成分と電子成分に分けることができます。電子成分は $\hat{H}_{\text{e}}\psi_{\text{e}}=(\hat{T}_{\text{e}} + \hat{V}_{\text{eN}} + \hat{V}_{\text{ee}})\psi_{\text{e}} = E_{\text{e}} \psi_{\text{e}}$ です。ボルン-オッペンハイマー近似（Born-Oppenheimer approximation）によれば、原子核の運動は電子の運動に比べて大変遅いため、その運動エネルギー $\hat{T}_{\text{N}}$ はゼロと近似できます。原子核引力項は単純なクーロン引力として計算できます。系の総エネルギーは、電子エネルギーと原子核同士の反発の和で求められます:

\begin{align}
E_{tot}=E_{el}+\sum_{A}\sum_{B>A}{\frac{Z_{A}Z_{B}}{R_{AB}}}
\end{align}

電子エネルギー $E_{el}$ を求めるために、***Hartree-Fock-Roothaan*** アプローチは、各電子を他の電子によって生成された平均場の中を移動する独立粒子として扱います。これらの電子は単一粒子のFock方程式に従います。

$$ \hat{f}_i \phi_i(\boldsymbol{r}_i) = \bigg(-\frac{1}{2} \nabla^2_i - \sum_{A} \frac{Z_A}{\boldsymbol{r}_{iA}} + \hat{v}_i^{\text{HF}}\bigg) \phi_i (\boldsymbol{r}_i)= \epsilon_i \phi_i (\boldsymbol{r}_i) $$

ここで、$\hat{v}_i^{\text{HF}}$ は電子間のクーロンおよび交換相互作用を考慮した有効平均場演算子です。これらの方程式を自己無撞着に解くことで、HF軌道 {$\phi_i(\boldsymbol{r}_i)$} と固有値 {$\epsilon_i $} が得られ、化学的に解釈できます。このようにして、電子ハミルトニアン $\hat{H}_{\text{e}}$ は行列方程式に簡略化されます。

Hohenberg と Kohn により、基底状態エネルギーを求めるためにSchr&ouml;dinger方程式を直接解く代わりに、電子密度を通じて求める強力な理論的基盤が作られました:

$$E[n(r)] = \int n(r) v_{\text{ext}}(r) dr + F[n(r)] $$

ここで、$F[n(r)]$ は未知の普遍汎関数であり、$n(r)$ は電子密度、$v_{\text{ext}}(r)$ は外部ポテンシャルです。***Density Functional Theory (DFT)*** 法はこれらの方程式を実用化し、非相互作用粒子の仮想系を導入して、単一粒子のハミルトニアン方程式に従わせます:

$$\hat{h}_{\text{KS}} \phi_i(r) = \Big[-\frac{1}{2} \nabla_i^2 + v_{\text{ext}}(r) + v_{\text{Ha}}(r) + v_{\text{XC}}(r) \Big] \phi_i(r) = \epsilon_i \phi_i(r)$$

ここで、$v_{\text{Ha}}(r)$ はハートリーポテンシャル、$v_{\text{XC}}(r)$ は交換-相関ポテンシャルです。この方程式の固有状態解を用いて、相互作用系の正確な密度を再現するように構築された密度を求めることができます。

# 計算練習

In [ ]:
# @title **1. 分子を選ぶ** { display-mode: "form" }

output.no_vertical_scroll() # avoids nested scrollbars

molecule_options = [('1. H2', 1), ('2. Water', 2), ('3. Methane', 3), ('4. Benzene', 4),
                    ('5. Custom molecule', 5)]

molecule_dropdown = widgets.Dropdown(description='分子:', options=molecule_options)

custom_smiles_input = widgets.Text(value='C1=NC2=NC=NC(=C2N1)N',
                                   description='SMILES: ',
                                   layout = widgets.Layout(width='450px'),
                                   disabled=True)

H2_r_select = widgets.BoundedFloatText(
    value=0.741,
    min=0.000,
    max=6.000,
    step=0.001,
    description='R(H2):',
    disabled=False,
    style={'description_width': 'initial'},
    layout = widgets.Layout(width='146px')
)

def on_dropdown_change(change):
    # Display or hide custom SMILES input based on dropdown selection
    if change['new'] == 5:
        custom_smiles_input.disabled = False
    else:
        custom_smiles_input.disabled = True
    # Display or hide bond length selection when H2 is chosen
    if change ['new'] > 1:
        H2_r_select.disabled = True
        H2_r_select.layout.display = 'none'
    else:
        H2_r_select.disabled = False
        H2_r_select.layout.display = 'block'

# Add observer to handle dropdown value changes
molecule_dropdown.observe(on_dropdown_change, names='value')

def render_molecule(molecule_num, custom_smiles, r_dist):
    # Displays 3D rendering of molecule optimized with simple force field.

    global molecule_smiles, molecule_name, mol, xyz_geom, h2
    mol1       = Chem.MolFromSmiles("[HH]"), 'Hydrogen dimer'
    mol2       = Chem.MolFromSmiles("O"), 'Water'
    mol3       = Chem.MolFromSmiles("C"), 'Methane'
    mol4       = Chem.MolFromSmiles("c1ccccc1"), 'Benzene'
    custom     = Chem.MolFromSmiles(custom_smiles), 'Custom molecule'

    # list of tuples of rdkit object (from Smiles) and molecule name
    molecule_smiles = [mol1, mol2, mol3, mol4, custom]

    molecule = molecule_smiles[molecule_num-1][0] # user-selected molecule from the list
    molecule_name = molecule_smiles[molecule_num-1][1] # molecule name

    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)

    if molecule_num == 1:
        xyz_geom = []
        x1, x2 = format(-r_dist/2, '.6f'), format(r_dist/2, '.6f')
        xyz_geom.append(f"H    {x1}    0.000000    0.000000")
        xyz_geom.append(f"H     {x2}    0.000000    0.000000")
        xyz_geom = "\n".join(xyz_geom)
        h2=f'''
     RDKit          3D

  2  1  0  0  0  0  0  0  0  0999 V2000
   {format(-r_dist/2, '.4f')}    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    {format(r_dist/2, '.4f')}    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0
M  END
$$$$'''
    else:
        AllChem.MMFFOptimizeMolecule(mol) # uses MMFF94 Force Field to optimize structure

        # format structure into readable xyz coordinates
        xyz_geom = []
        for lines in Chem.MolToXYZBlock(mol).split("\n")[2:]:
            strip = lines.strip()
            if strip:
                xyz_geom.append(strip)
        xyz_geom = "\n".join(xyz_geom)

    # display .xyz coordinates of structure
    print('\n 選択した分子の(x,y,z)座標 (\u00C5): \n')
    print(xyz_geom)
    print(' ')

    # render 3D ball-and-stick model of structure
    if molecule_num == 1:
        view = py3Dmol.view(
        data=h2,
        style={"stick": {}, "sphere": {"scale": 0.3}},
        width=400,
        height=300,
        )
    else:
        view = py3Dmol.view(
        data=Chem.MolToMolBlock(mol),
        style={"stick": {}, "sphere": {"scale": 0.3}},
        width=400,
        height=300,
        )
    view.zoomTo()
    view.show()

### setting up the display ###
output_display = widgets.interactive_output(render_molecule, {'molecule_num':molecule_dropdown, 'custom_smiles':custom_smiles_input, 'r_dist': H2_r_select})
panel = widgets.HBox([molecule_dropdown, H2_r_select])
top_panel = widgets.VBox([panel,  custom_smiles_input])
display(widgets.VBox([top_panel, output_display]))

In [ ]:
# @title **2. SCF 計算** { display-mode: "form" }

output.no_vertical_scroll() # avoids nested scrollbars

### scf widgets configure ###
basis_options = [('STO-3G', 1), ('3-21G', 2), ('6-31G', 3), ('6-31+G*', 4),
                 ('Other basis', 5)]
basis_dropdown = widgets.Dropdown(description='基底関数系を選んで: ', options=basis_options,
                                  layout=widgets.Layout(width='auto'), style={'description_width': 'initial'})
custom_basis_input = widgets.Text(value='ccpvtz',
                                   description='他の基底関数系: ', style={'description_width': 'initial'},
                                   layout = widgets.Layout(width='240px'),
                                   disabled=True)
def on_dropdown_change(change):
    # Display or hide custom basis input based on dropdown selection
    if change['new'] == 5:
        custom_basis_input.disabled = False
    else:
        custom_basis_input.disabled = True
# Add observer to handle dropdown value changes
basis_dropdown.observe(on_dropdown_change, names='value')

def run_calculation(basis_num, custom_basis, method, xyz_struc=xyz_geom):
    '''
    Performs a HF single-point calculation on a molecule.
    Energy results are displayed in a Markdown table.
    Args:
      xyz_struc : str Cortesian coordinates
      basis_num : int index in basis dropdown
      custom_basis : str custom basis input
    '''
    global basis_sets
    basis1       = 'sto3g', 'STO-3G'
    basis2       = '321g', '3-21G'
    basis3       = '631g', '6-31G'
    basis4       = '631+g*', '6-31+G*'
    custom       = custom_basis, 'Custom Basis'

    # list of tuples of pyscf basis definition and name
    basis_sets = [basis1, basis2, basis3, basis4, custom]
    basis = basis_sets[basis_num-1][0] # user-selected basis from the list

    global mf, mol_quantum
    # create pyscf object from input structure
    mol_quantum = gto.M(atom=xyz_struc, basis=basis, unit="ANG")
    mol_quantum.build()
    if method == 'hf':
        print(f"\n HF一点計算の実行中 \u231B ...")
        mf = scf.RHF(mol_quantum).run(verbose=4)
    elif method == 'dft':
        print(f"\n LDA汎関数によるDFT一点計算の実行中 \u231B ...")
        mf = dft.RKS(mol_quantum).run(verbose=4)
    print("\n \033[1m SCFが終わり! \033[0m \n")

    mf.dump_scf_summary()

    mos        = mf.mo_coeff
    elec_ener  = mf.energy_elec()[0] # electronic energy
    nuc_ener   = mf.energy_nuc()     # nuclear     "
    total_ener = mf.energy_tot()     # total       "
    # Markdown table display:
    display(Markdown('<br>'))
    display(Markdown(f'''成分|エネルギー (a.u.)
    :---:|:---:
    電子エネルギー  | {elec_ener:.3f}
    核エネルギー    | {nuc_ener:.3f}
    **全エネルギー**| {total_ener:.3f}
    '''))
    display(Markdown('<br>'))

### electronic button configure ###
properties_button = widgets.Button(description='分子特性',
                                   layout=widgets.Layout(width='auto'))
properties_button.style.font_weight = 'bold'

def elec_properties(b):
  '''
  Prints dipole moment and Mulliken charges from HF calculation result.
  Args:
    b : Button click.
  '''
  mf.dip_moment(verbose=3)
  print("\n===============================")
  mf.mulliken_pop(verbose=3)
  print(" ")
  for atom in mol.GetAtoms():
    atom.SetProp('molAtomMapNumber', str(atom.GetIdx()+1))
  display(mol)
  properties_button.disabled = True

properties_button.on_click(elec_properties) # link button + function

### scf button configure ###
scf_button = widgets.Button(description='HF-SCFを実行する',layout=widgets.Layout(width='auto'))
scf_button.style.font_weight = 'bold'

### dft button configure ###
dft_button = widgets.Button(description='DFT(LDA)-SCFを実行する',layout=widgets.Layout(width='auto'))
dft_button.style.font_weight = 'bold'

def click_scf(b):
  '''Runs the HF calculation.
  Args:
    b : button click.
  '''
  global calc_method
  calc_method = 'HF'
  run_calculation(basis_dropdown.value, custom_basis_input.value, 'hf', xyz_geom)
  display(Markdown('---'))
  display(properties_button)
  display(Markdown('<br>'))
  scf_button.disabled = True
  basis_dropdown.disabled = True
  custom_basis_input.disabled = True
  dft_button.disabled = True
  properties_button.disabled = False

scf_button.on_click(click_scf) # link button + function

def click_dft(b):
  '''Runs the DFT calculation.
  Args:
    b : button click.
  '''
  global calc_method
  calc_method = 'DFT'
  run_calculation(basis_dropdown.value, custom_basis_input.value, 'dft', xyz_geom)
  display(Markdown('---'))
  display(properties_button)
  display(Markdown('<br>'))
  scf_button.disabled = True
  basis_dropdown.disabled = True
  custom_basis_input.disabled = True
  dft_button.disabled = True
  properties_button.disabled = False

dft_button.on_click(click_dft) # link button + function

### setting up the display ###
panel = widgets.HBox([basis_dropdown, scf_button, dft_button])
top_panel = widgets.VBox([panel,  custom_basis_input])
display(top_panel)
basis_link = "[すべての基底関数系](https://pyscf.org/_modules/pyscf/gto/basis.html)"
print(basis_link)

In [ ]:
# @title **3. 分子軌道のダイアグラム** { display-mode: "form" }

output.no_vertical_scroll() # avoids nested scrollbars

# assert 'mf' in globals(), 'Please run the above HF calculation first.'

def mo_diagram(mf):
    '''Displays MO energy level diagram.
    Args:
      mf (pyscf.scf.RHF): pyscf HF calculation object.
    '''
    # Find HOMO-LUMO gap
    homo_num  = np.count_nonzero(mf.mo_occ == 2) - 1
    lumo_num  = np.count_nonzero(mf.mo_occ == 2)
    homo_en   = mf.mo_energy[homo_num]
    mos_occ   = mf.mo_coeff[:,:lumo_num]
    lumo_en   = mf.mo_energy[lumo_num]
    mos_unocc = mf.mo_coeff[:,lumo_num:]
    hl_gap    = abs(lumo_en - homo_en)
    hl_gap_ev = hl_gap * 27.2114

    # Create a list MO labels
    MO_labels = []
    for i in range(len(mf.mo_occ)-lumo_num):
        if i == 0:
            MO_labels.append('LUMO')
        else:
            MO_labels.append(f'LUMO+{i}')
    MO_labels.reverse()
    for i in range(homo_num+1):
        if i == 0 :
            MO_labels.append('HOMO')
        else:
            MO_labels.append(f'HOMO-{i}')
    MO_labels.reverse()
    #print(MO_labels)

    table = pd.DataFrame({"En (a.u.)": mf.mo_energy, "En (eV)": 27.2114 * mf.mo_energy, "Occ.": mf.mo_occ.astype('int'), "Type": MO_labels})
    table = table.round(decimals=3)
    #print(table)

    print(f'MOの総数 = ', table.shape[0])
    print('最大8つフロンティアMOのダイアグラムを作る ......')

    global frontier_nums, frontier_labels, frontier_energies

    homo_front_min    = homo_num if homo_num < 3 else homo_num - 3
    lumo_front_max    = table.shape[0] if lumo_num + 4 > table.shape[0] else lumo_num + 4
    frontier_nums     = np.arange(homo_front_min,lumo_front_max,1)
    frontier_labels   = table.iloc[frontier_nums]['Type'].tolist()
    frontier_energies = table.iloc[frontier_nums]['En (a.u.)'].tolist()

    y = frontier_energies # orbital energy values
    x = [1] # orbital positions on the graph (useful for degenrated orbitals)
    for i in range(1,len(y)):
        if y[i] == y[i-1]:
          x.append(x[i-1]+0.3)
        else:
          x.append(1)

    # Building up the diagram
    fig, ax = plt.subplots(figsize=(10, 6))
    plt.ylabel("Energy (a.u.)", size=11)
    plt.scatter(x[:homo_num - homo_front_min + 1], y[:homo_num - homo_front_min + 1], marker=0, s=1000, linewidths=2, color='orange', label='occupied')
    plt.scatter(x[homo_num - homo_front_min + 1:], y[homo_num - homo_front_min + 1:], marker=0, s=1000, linewidths=2, color='green', label='unoccupied')
    for xi, yi, labels in zip(x, y, frontier_labels):
        plt.annotate(labels, (xi, yi), size=8, xycoords='data', xytext=(5, -3), textcoords='offset points')
    for xi, yi, labels in zip(x, y, y):
        plt.annotate(labels, (xi, yi), size=8, xycoords='data', xytext=(-70, -3), textcoords='offset points')
    plt.rcParams["legend.markerscale"] = 0.50
    plt.legend(loc='upper right', handletextpad=-0.3)
    plt.title(f'Frontier Orbitals')
    plt.xticks([])

    plt.xlim([min(x) - 0.3, max(x) + 0.3])
    #plt.ylim([min(y) - 0.5, max(y) + 0.5])

    # display HOMO-LUMO gap value
    plt.text(0.8, 0.05, f'HOMO-LUMO Gap: {hl_gap:.3f} a.u.\n({hl_gap_ev:.3f} ev)',
             horizontalalignment='center', verticalalignment='center',
             transform=ax.transAxes, size=11)
    plt.show()

    display(table.iloc[homo_front_min:lumo_front_max][::-1])

mo_diagram(mf)



In [ ]:
# @title　**4. Cubeファイルの生成**　{ display-mode: "form" }

# %%capture prevents output from displaying
%%capture cap --no-stderr

good_isovalues = [] # list of isovalues that render well in subsequent visualization

def get_orbital(i, label):
  '''Saves .cube files of orbital and prob. density from the above calculation.
  Args:
    i (int): Index to iterate over in for loop.
    label (str): MO label, i.e. HOMO, HOMO-1, etc.
  '''
  tools.cubegen.orbital(mol_quantum, f'{label}.cube', mf.mo_coeff[:,i],nx=80,ny=80,nz=80)

print(".cube ファイルの生成 (~1分　かかる) \u231B ...", end= "\n\n", file=sys.stderr)
# Generate .cube files for all orbitals in the above table
for i in range(len(frontier_nums)):
  get_orbital(frontier_nums[i],frontier_labels[i])
print("\033[1m終わり! \033[0m", end='\n\n', file=sys.stderr)

# save data to dictionary
grid_data = {}
for name in frontier_labels:
  with open(f'{name}.cube') as f:
    grid_data[f'{name}'] = f.read()

In [ ]:
# @title **5. 3D分子軌道のプロット** { display-mode: "form" }


output.no_vertical_scroll() # avoids nested scrollbars

orbital_select = widgets.Dropdown(value='HOMO',options=frontier_labels,description='分子軌道: ')

isoval_select = widgets.BoundedFloatText(
    value=0.02,
    min=0,
    max=1.0,
    step=0.01,
    description='アイソバリュー:',
    disabled=False,
    style={'description_width': 'initial'}
)

def draw_orbital(label, isoval, show_noninteractive_png=False):
  '''Renders isosurface of selected orbital from calculation.
  Args:
    label (str): Orbital designation, e.g. HOMO, HOMO-1, etc.
  '''
  view = py3Dmol.view(width=500,height=500)
  print('----------------------------------------')
  #print(f'isovalue: {isoval:.4f}')
  print('レンダリング中 (~30秒　かかる) \u231B ...')
  view.addVolumetricData(grid_data[f'{label}'], "cube", {'isoval': isoval, 'color': "red", 'opacity': 0.90})
  view.addVolumetricData(grid_data[f'{label}'], "cube", {'isoval': -isoval, 'color': "blue", 'opacity': 0.90})
  if molecule_name == 'Hydrogen dimer':
    view.addModel(h2, 'mol')
  else:
    view.addModel(Chem.MolToMolBlock(mol), 'mol')
  view.setStyle({'stick':{}, 'sphere': {"scale":0.3}})
  if show_noninteractive_png:
    view.zoomTo()
    view.show()
    view.png()
  else:
    view.zoomTo()
    view.show()

#display(Markdown(f'### {molecule_smiles[molecule_dropdown.value-1][1]} frontier MOs calculated at {calc_method}/{basis_sets[basis_dropdown.value-1][0]} level of theory'))
#print(" ")

widgets.interactive(draw_orbital, {'manual': True, "manual_name": "レンダリングする"}, label=orbital_select, isoval = isoval_select)

# 練習問題
***6つのタスクが提案されています。少なくとも3つを選び、その結果をレポートにまとめてください。***

1. 提示されたリストにない分子の全エネルギー・双極子モーメント・原子の電荷・HOMO-LUMOギャップを計算してください。次のURLページに分子名を入力してSMILES定義を検索できます: https://pubchem.ncbi.nlm.nih.gov
2. 任意の分子を選び、同じ方法(HFまたは DFTのどちらかを選択)で、異なる基底関数（最低でも3種類）で計算してください。全エネルギーと原子の電荷とHOMO-LUMOギャップを比較してください。得られたHOMO-LUMOギャップが最も実験値に近いのはどの基底関数を用いた場合かを考察してください
3. 任意の分子を選び、同じ方法(HFまたは DFTのどちらかを選択)で、異なる基底関数（最低でも3種類）で計算してください。基底関数の違いによりSCFが収束するまでに何回の反復が必要であったかを考察してください
4. HFまたは DFTのどちらかを選択し、いづれかの基底関数を用いてmethanとbenzeneを計算し、MOsを可視化してください。縮退したMO(同じエネルギーのMOs)の3Dプロットを比較してください。縮退MOsが現れる理由を考察してください。
5. H2分子を選択し、H-H間距離が1.2Angstromから0.4 Angstrom まで、0.2Angstrom間隔で変化させ、HH核間距離の変化に対して次の事を調べ考察してください；1)MOエネルギーの変化、2)全エネルギーの変化、3)核間反発エネルギーの変化、4) 1電子エネルギーの変化、5) 2電子エネルギーの変化。なお、HFまたは DFTのどちらを選択してもよい。
6. 上記の５つのタスクにはない別の問題を自分で考え、それを実行し結果を考察してください。レポートの際、どういう問題を考えたかを最初に書くこと。




# 参考文献

* Hirschi, J. S.; Bashirova, D.; Zuehlsdorff, T. J. Opening the Density Functional Theory Black Box: A Collection of Pedagogic Jupyter Notebooks. J. Chem. Educ. 2023, 100 (11), 4496-4503.
* Recent developments in the PySCF program package, Q. Sun et.al. J. Chem. Phys. 153, 024109 (2020) | https://pyscf.org/index.html
* "Modern Quantum Chemistry: Introduction to Advanced Electronic Structure Theory" by Attila Szabo & Neil S. Ostlund

© 2024 Mariia IVONINA (Tada Lab. at Kyushu University, Q-PIT). 無断転載を禁じます。